In [ ]:
%matplotlib inline

# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

## 一、文件夹结构设计和数据存放

### 1. 建立存放数据的文件夹

In [46]:
# shutil －一种高层次的文件操作工具，强大之处是在对文件的复制与删除操作支持比较好
import os
import shutil

In [7]:
# 建立data文件夹用于存放数据，train和val分别存放训练集和验证集
# .mkdir的函数用于建立该路径下的文件夹
os.mkdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/data')
os.mkdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/data/train')
os.mkdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/data/val')

In [9]:
# 分别建立存放小狗图片的文件夹和存放小猫图片的文件夹
# os.path.join用于路径拼接
train_root = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/data/train'
dog_folder = os.path.join(train_root, 'dog')
cat_folder = os.path.join(train_root, 'cat')
os.mkdir(dog_folder)
os.mkdir(cat_folder)

In [10]:
# 大致同上
val_root = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/data/val'
dog_folder = os.path.join(val_root, 'dog')
cat_folder = os.path.join(val_root, 'cat')
os.mkdir(dog_folder)
os.mkdir(cat_folder)

### 2. 分割数据集并将其存入对应文件夹


In [42]:
# .listdir主要用于列出该路径下所有的文件的名称，以列表的形式得到
# 这里的路径是# 下载下来的猫狗大战的训练集数据存放的路径
data_file = os.listdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/zip/train')

In [43]:
# 用filter分别得到是小狗的文件名和是小猫的文件名的列表
dog_file = list(filter(lambda x: x[:3]=='dog', data_file))
cat_file = list(filter(lambda x: x[:3]=='cat', data_file))

In [1]:
# 分割小狗的数据集，如果i < len(dog_file)*0.9，那么就将其放到用来存放数据集的文件夹下的训练集的小狗文件夹中
# 否则便存放在验证集
root = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/PlayGround/kaggle_dog_vs_cat/'
for i in range(len(dog_file)):
    pic_path = root + 'zip/train/' + dog_file[i]
    if i < len(dog_file)*0.9:
        obj_path = train_root + '/dog/' + dog_file[i]
        print(obj_path)
    else:
        obj_path = val_root + '/dog/' + dog_file[i]
    shutil.copy(pic_path, obj_path)

In [2]:
# 大致同上
for i in range(len(cat_file)):
    pic_path = root + 'zip/train/' + cat_file[i]
    if i < len(dog_file)*0.9:
        obj_path = train_root + '/cat/' + cat_file[i]
        print('i:',i  ,obj_path)
    else:
        obj_path = val_root + '/cat/' + cat_file[i]
    shutil.move(pic_path, obj_path)

### 通过以上代码，得到数据目录结构为：
+ train
  + dog
  + cat
    
+ val
  + dog
  + cat
+ test
  + test
  

## 二、数据增强与加载

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/dogs-vs-cats/'

# 使用datasets.ImageFolder分别加载训练集、验证集和测试集的数据
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}

# 得到可batch的数据格式
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
testdataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=False, num_workers=4)
              for x in ['test']}

# 得到数据规模
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}

# 得到分类
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 三、加载模型

目前来说，大多数的图片训练都不会从零开始。一方面是机子性能的原因，另一方面是由于大多数数据集的数据量普遍不足以从零训练一个比较完美的分类器，所以，一般会选择用迁移学习的方式在训练一个模型

In [ ]:
# 得到一个resnet18已经训练过的网络（pretrained=True表示已经训练过）
model_ft = models.resnet18(pretrained=True)
# 得到输出层前的层数
num_ftrs = model_ft.fc.in_features
# 将输出层换成2
model_ft.fc = nn.Linear(num_ftrs, 2)
# 设置为GPU处理
model_ft = model_ft.to(device)

# 设置损失函数
criterion = nn.CrossEntropyLoss()
# 设置优化算法
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

<b>注意：</b> 训练模型期望的输入是RGB图像的mini-batch：(batch_size, 3, H, W)，并且H和W不能低于224。图像的像素值必须在范围[0,1]间，并且用均值mean=[0.485, 0.456, 0.406]和方差std=[0.229, 0.224, 0.225]进行归一化。

## 四、训练模型

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    # static_dict()用于返回包含模块所有状态的字典，包括参数和缓存。键是参数名称或者缓存名称。
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # 加入一个batch，将得到的数据放置到制定的设备上（GPU或者CPU）
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 清空梯度
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)

## 五、可视化

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_ft)

## 六、测试集和提交

In [ ]:
# 生成结果存放在preds中
preds = []
for i, (inputs, labels) in enumerate(testdataloaders['test']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model_ft(inputs)
    preds.append(torch.max(outputs, 1)[1])

In [ ]:
# 将字典转化成numpy格式，以方便上传
labels=[]
for i in range(len(preds)):
    labels.extend(preds[i].cpu().numpy())

In [ ]:
# 得到图片的序号
fileindex = []
for i in range(12500):
    fileindex.append(testdataloaders['test'].dataset.imgs[i][0].split('\\')[-1].split('.')[0])

In [ ]:
# 组合序号和预测结果，存放在.csv中
import pandas as pd

result = pd.Series(labels,index=fileindex)
result.to_csv('submit.csv')

## 七、模型的保存和加载

In [ ]:
# 模型保存
torch.save(model_ft, 'dogsvscat.pkl')

In [ ]:
# 模型加载
model_ft = torch.load('dogsvscat.pkl')    